### Исследование таблицы Сочетаемость БАД и лекарства

In [1]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel(
    "./data/new_vitamins.xlsx", sheet_name="Сочетаемость БАД и лекарства"
)

In [3]:
df.sample(3)

,ID Взаимодействия,ID Ингредиента БАДа,ID Лекарства,Описание Взаимодействия,Влияние на Усвояемость
4280,I000004281,V000000010,D00140,Пиридоксин может снижать скорость выведения аз...,негативное
122,I000000123,V000000014,D00107,Метаболизм витамина D может снижаться при соче...,негативное
1913,I000001914,V000000008,D01662,Концентрация трастузумаба эмтанзина в сыворотк...,позитивное


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7725 entries, 0 to 7724
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID Взаимодействия        7725 non-null   object
 1   ID Ингредиента БАДа      7725 non-null   object
 2   ID Лекарства             7725 non-null   object
 3   Описание Взаимодействия  7725 non-null   object
 4   Влияние на Усвояемость   7725 non-null   object
dtypes: object(5)
memory usage: 301.9+ KB


В БД в данный момент собрано 7725 комбинаций лекарственных препаратов с витаминами и минералами. Описаны эффекты от их взаимодействия, определен тип влияния на усвояемость - позитивный или негативный.

На данном этапе эта таблица интересна с позиции распределения позитивного и негативного эффекта от сочетания лекарства и ингредиента.

In [5]:
df["Влияние на Усвояемость"].value_counts(normalize=True)

Влияние на Усвояемость
негативное    0.842589
позитивное    0.157411
Name: proportion, dtype: float64

84% сочетаний имеют негативный эффект.

In [6]:
df["Описание Взаимодействия"].value_counts()

Описание Взаимодействия
Метаболизм Дувелисиба может снижаться при сочетании с ниацином.                                                                                                                    2
Риск или тяжесть миопатии, рабдомиолиза и миоглобинурии могут повышаться при сочетании ниацина с тербинафином.                                                                     2
Метаболизм прометазина может снижаться при сочетании с ниацином.                                                                                                                   2
Метаболизм тенофовира алафенамида может снижаться при сочетании с ниацином.                                                                                                        2
Метаболизм дулоксетина может снижаться при сочетании с ниацином.                                                                                                                   2
                                                                       

Как видно из посчета количества вхождений, очень мало описаний взаимодействия совпадают. Для текущего вида БД можно оставить эти данные в таком виде. Но для дальнейшего расширения нужна формализация этих описаний.

In [7]:
df["ID Ингредиента БАДа"].value_counts(normalize=True)

ID Ингредиента БАДа
V000000008    0.119741
V000000006    0.101877
V000000025    0.094757
V000000012    0.090874
V000000132    0.083883
V000000019    0.079482
V000000188    0.077799
V000000010    0.076634
V000000016    0.050485
V000000034    0.048803
V000000017    0.042848
V000000014    0.025243
V000000038    0.022783
V000000037    0.016570
V000000033    0.012945
V000000039    0.011909
V000000005    0.008544
V000000009    0.007767
V000000063    0.007120
V000000004    0.005955
V000000013    0.005566
V000000007    0.004272
V000000041    0.003495
V000000032    0.000259
V000000020    0.000129
V000000028    0.000129
V000000024    0.000129
Name: proportion, dtype: float64

Максимальное количество сочетаний описано в БД для витаминов B3(11.97%)б  В12(10,18%), L-карнитина(9,47%), В12(9,08%) и генистеина(8,38%).

In [8]:
df["ID Лекарства"].value_counts()

ID Лекарства
D00431    16
D01063    14
D01739    14
D00264    14
D00405    14
          ..
D00645     1
D00654     1
D01139     1
D00690     1
D01602     1
Name: count, Length: 1761, dtype: int64

Больше всего сочетаний описано для Ингредиента V000000008 (B3) и лекарства ID Лекарства D00431 (Циклоспорин). Посмотрим, как часто эти два признака встречаются вместе и как влияют друг на друга.

In [9]:
df[(df["ID Ингредиента БАДа"]=='V000000008') & (df["ID Лекарства"]=="D00431")]

,ID Взаимодействия,ID Ингредиента БАДа,ID Лекарства,Описание Взаимодействия,Влияние на Усвояемость
420,I000000421,V000000008,D00431,Концентрация циклоспорина в сыворотке крови мо...,позитивное
430,I000000431,V000000008,D00431,Концентрация циклоспорина в сыворотке крови мо...,позитивное


Необходимо более подробно исследовать сочетания.